<a href="https://colab.research.google.com/github/Areeb-07/ChatBot-2/blob/main/chatter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data = pd.read_csv('/content/drive/MyDrive/chatbot/Data.csv')

In [5]:
training_data=data.values

In [6]:
X = training_data[:,:-1]
X = [a[0] for a in X]
Y = training_data[:,-1]

In [7]:
oh = np.eye(9)

In [8]:
Y_oh = [oh[i] for i in Y]

In [9]:
import csv
words = pd.read_table('/content/drive/MyDrive/chatbot/glove.6B.200d.txt', sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE)

In [10]:
def word_2_vec(word):
  global words
  return words.loc[word].values
def index_to_word(index):
  global words
  return words.index.values[index]
def word_to_index(word):
  global words
  return int(list(words.index.values).index(word))

In [11]:
import string 
result = string.punctuation 
punctuations = [i for i in result]

In [20]:
def sentences_to_indices(X, max_len):
 
    m = len(X)                                
    
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                             
        
        sentence_words = []
        for k in X[i].split():
          found = False
          ind = None
          for l in k:
              if l in punctuations:
                found = True
                ind = k.index(l)
                break
          if found:
              sentence_words.append(k[:ind].lower())
              sentence_words.append(k[ind:].lower())
              continue
          sentence_words.append(k.lower())      
        j = 0

        for w in sentence_words:
            X_indices[i, j] = int(word_to_index(w))
            j += 1    
        int_array = X_indices.astype(int)
    return int_array

In [13]:
def pretrained_embedding_layer():
    vocab_len = len(words.index.values) + 1                  
    emb_dim = word_2_vec("cucumber").shape[0]     

    emb_matrix = np.zeros((vocab_len,emb_dim))

    for index in range(vocab_len-1):
      if index_to_word(index) == index_to_word(index):
        embedding_vector = word_2_vec(index_to_word(index))
        if embedding_vector is not None:
          emb_matrix[index, :] = embedding_vector

    embedding_layer = Embedding(vocab_len, emb_dim,trainable=False)

    embedding_layer.build((None,))

    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [14]:
def chatbot(input_shape):

    sentence_indices = Input(shape = input_shape, dtype = 'int32')

    embedding_layer = pretrained_embedding_layer()

    embeddings = embedding_layer(sentence_indices)   

    X = LSTM(128, return_sequences=True)(embeddings)

    X = Dropout(0.5)(X)

    X = LSTM(128, return_sequences=False)(X)

    X = Dropout(0.5)(X)

    X = Dense(9)(X)

    X = Activation("softmax")(X)
    model = Model(inputs=sentence_indices, outputs=X)

    return model

In [15]:
model = chatbot((10,))
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 10)]              0         
                                                                 
 embedding (Embedding)       (None, 10, 200)           80000200  
                                                                 
 lstm (LSTM)                 (None, 10, 128)           168448    
                                                                 
 dropout (Dropout)           (None, 10, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 9)                 1161  

In [16]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
X_train_indices = sentences_to_indices(X, 10)

In [26]:
Y_train = np.array(Y_oh)

In [30]:
model.fit(X_train_indices, Y_train, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
2/2 [==============================] - 7s 22ms/step - loss: 2.1869 - accuracy: 0.1064
Epoch 2/50
2/2 [==============================] - 0s 13ms/step - loss: 2.0970 - accuracy: 0.2766
Epoch 3/50
2/2 [==============================] - 0s 13ms/step - loss: 2.0296 - accuracy: 0.3617
Epoch 4/50
2/2 [==============================] - 0s 13ms/step - loss: 1.9961 - accuracy: 0.3617
Epoch 5/50
2/2 [==============================] - 0s 12ms/step - loss: 1.9160 - accuracy: 0.3617
Epoch 6/50
2/2 [==============================] - 0s 13ms/step - loss: 1.8797 - accuracy: 0.3404
Epoch 7/50
2/2 [==============================] - 0s 9ms/step - loss: 1.7190 - accuracy: 0.4255
Epoch 8/50
2/2 [==============================] - 0s 12ms/step - loss: 1.5208 - accuracy: 0.4894
Epoch 9/50
2/2 [==============================] - 0s 12ms/step - loss: 1.4121 - accuracy: 0.5532
Epoch 10/50
2/2 [==============================] - 0s 12ms/step - loss: 1.3621 - accuracy: 0.5319
Epoch 11/50
2/2 [=============

In [31]:
def label_to_text(index):
  text=["Hello, how can I help you?", "I am good. How about you?", "Nice to meet you too", "Have a great day", "Goodbye. See you soon", "Good morning", "Good afternoon", "Good evening", "Good Night"]
  return text[int(index)]

In [34]:
def chat():
  print("Chat with the chatbot. Type q to exit")
  user=input()
  while user.lower()!='q':
    x_test = np.array([user])
    X_test_indices = sentences_to_indices(x_test, 10)
    print(label_to_text(np.argmax(model.predict(X_test_indices))))
    user=input()

In [37]:
chat()

Chat with the chatbot. Type q to exit
Hello
Hello, how can I help you?
How you doing?
I am good. How about you?
Nice meeting you
Nice to meet you too
Bye Bye
Goodbye. See you soon
Afternoon
Good morning
Have a good afternoon
Good morning
Good night
Good Night
Have a wonderful day
Have a great day
See you
Goodbye. See you soon
q


In [38]:
model.save("/content/drive/MyDrive/chatbot/chat_model")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/chatbot/chat_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/chatbot/chat_model/assets
